## **05-2) 교차 검증과 그리드 서치**

- 검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 배운다.
- 그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터를 찾는다.

#### **용어 정리**
- **검증 세트**: 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트
- **교차 검증**: 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련한다. 교차검증은 이런 식으로 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법이다.
- **그리드 서치**: 하이퍼파라미터 탐색을 자동화해 주는 도구. 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택한다. 마지막으로 이 매개변수 조합으로 최종 모델을 훈련한다.
- **랜덤 서치**: 연속된 매개변수 값을 탐색할 때 유용하다. 탐색할 값을 직접 나열하는 것이 아니고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달한다. 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있다.

<hr><br>

최적의 모델을 위한 하이퍼파라미터를 탐색하려면 다양한 하이퍼파라미터를 시도해 봐야 한다.<br>
이런 과정에서 테스트 세트를 사용하면 결국 테스트 세트에 맞춰 모델을 훈련하는 효과를 만들게 된다.<br>


테스트 세트는 최종 모델을 선택할 때까지 사용하지 말아야 한다.<br>


### **검증 세트**(validation set)

테스트 세트를 사용하지 않고 모델을 평가하려면 또 다른 세트인 **검증 세트**가 필요하다.<br>
혹은 개발 세트(dev set)라고도 하는데, 검증 세트는 훈련 세트 중 일부를 다시 덜어내 만든다.

검증 세트가 추가된 과정을 정리하자면 다음과 같다.
1. 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다.<br>
2. 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고르고,<br>이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다.<br>
3. 그리고 마지막에 테스트 세트에서 최종 점수를 평가한다.

그럼 검증 세트를 만들어보자. train_test_split() 함수를 2번 적용해서 나눠주면 된다.

In [2]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

훈련 세트와 검증 세트의 크기를 확인해보자.

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


이제 sub_input, val_input, sub_target, val_target을 사용해 모델을 만들고 평가해볼 것이다.

In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈련 세트에 과대적합된 것을 알 수 있다. 매개변수를 바꿔서 더 좋은 모델을 찾아야 할 것이다.

<br><hr><br>

### **교차 검증**(cross validation)

검증 세트가 크지 않다면 어떻게 데이터를 나눴는지에 따라 검증 점수가 들쭉날쭉하고 불안정할 것이다.<br>
**교차 검증**을 하면 안정적인 검증 점수를 얻고, 훈련에 더 많은 데이터를 사용할 수 있다.

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한 뒤, 이 점수를 평균하여 최종 검증 점수를 얻는다.<br>


보통 훈련 세트를 5등분 혹은 10등분 하는데, 나누어진 한 덩어리를 **폴드**라고 한다.<br>
한 폴드씩 돌아가면서 검증 세트의 역할을 한다. 따라서 전체적으로 5개 혹은 10개의 모델을 만든다.

사이킷런의 `cross_validate()`라는 교차 검증 함수를 사용해보자.

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
# scores = cross_val_score(dt, train_input, train_target) ~> test_score만 반환함
print(scores)

{'fit_time': array([0.02448511, 0.0245347 , 0.02514529, 0.01461244, 0.0104444 ]), 'score_time': array([0.00128889, 0.00114799, 0.00112033, 0.00119162, 0.00118232]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


fit_time 키는 모델을 훈련하는 시간, score_time 키는 모델을 검증하는 시간을 의미하며, 각 키마다 5개의 숫자가 담겨있다. <br>
cross_validate()는 기본적으로 5-폴드 교차 검증을 수행하기 때문이다. 폴드 수는 cv 매개변수로 지정할 수 있다.

교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.<br>
이름은 test_score지만 검증 폴드의 점수임을 유의하자.

In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


한 가지 주의할 점은 cross_validate()는 폴드를 나눌 때 훈련 세트를 섞지 않는다.<br>
우리는 앞서 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없었지만, <br>만약 교차 검증을 할 때 훈련 세트를 섞으려면 **분할기**(splitter)를 지정해야 한다.

cross_validate()는 기본적으로 
- 회귀모델일 경우 KFold 분할기를 사용하고,
- 분류모델일 경우 StratifiedKFold 분할기를 사용한다.

앞서 수행한 교차 검증은 다음 코드와 동일하다.

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성한다.

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


<br><hr><br>

### **하이퍼파라미터 튜닝**

+) **하이퍼파라미터**는 사용자 지정 파라미터이다.

교차 검증을 사용해 다양한 하이퍼파라미터를 탐색한다. 머신러닝 라이브러리에서는 클래스와 메서드의 매개변수를 바꾸어 모델을 훈련하고 평가해보는 작업이다.<br>

사이킷런의 **그리드 서치**를 사용하면 테스트하고 싶은 매개변수 리스트를 만들어 위 과정을 자동화할 수 있다.

#### **그리드 서치**

사이킷런의 `GridSearchCV` 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 별도로 cross_validate() 함수를 호출할 필요가 없다.

<br>

기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾아볼 것이다.<br>
먼저 GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만든다.

In [13]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다.

In [15]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs 객체에 fit() 메서드를 호출한다. <br>
이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행한다.

- GridSearchCV의 cv 매개변수 기본값은 5이고, min_impurity_decrease 값마다 5-폴드 교차 검증을 수행하므로 5*5=25개의 모델을 훈련한다.
- n_jobs에서는 병렬 실행에 사용할 CPU 코어 수를 지정한다. 기본값은 1이며, -1로 지정하면 시스템에 있는 모든 코어를 사용한다.

In [16]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중<br>
검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.<br>
이 모델은 gs 객체의 `best_estimator_`속성에 저장되어 있다.

In [17]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 `best_params_` 속성에 저장되어 있다.

In [18]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 `cv_results_` 속성의 `mean_test_score`키에 저장되어 있다. <br>
5번의 교차 검증으로 얻은 점수를 출력해보자.

In [20]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


넘파이 `argmax()`함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.<br>
이 인덱스를 사용해 `params` 키에 저장된 매개변수를 출력할 수 있다. 이 값이 최상의 검증 점수를 만든 매개변수 조합이다.<br>
앞서 출력한 gs.best_params와 동일한지 확인해보자.

In [21]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


그럼 이 과정들을 정리해보자.
1. 먼저 탐색할 매개변수를 지정한다.
2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. <br> 이 조합은 그리드 서치 객체에 저장된다.
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

조금 더 복잡한 매개변수 조합을 탐색해보자.
- min_impurity_decrease: 노드를 분할하기 위한 불순도 감소 최소량
- max_depth: 트리 깊이 제한
- min_samples_split: 노드를 나누기 위한 최소 샘플 수

In [27]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

이 매개변수로 수행할 교차 검증 횟수는 9 * 15 * 10 = 1,350개이다.<br>
기본 5-폴드 교차검증을 수행하므로 만들어지는 모델의 수는 6,750개가 된다.

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

최상의 매개변수 조합을 확인해보자.

In [29]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검증 점수도 확인해보자.

In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


<br><hr><br>

#### **랜덤 서치**

매개변수 값이 수치형이고 특히 연속적인 실숫값이라면 **랜덤 서치**를 사용하는 게 좋다.<br>
**랜덤 서치**에서는 매개변수를 샘플링할 수 있는 싸이파이의 확률 분포 객체를 전달하여 특정 범위 내에서 지정된 횟수만큼 <br>매개변수 후보 값을 샘플링하여 교차 검증을 시도할 수 있다.<br><br>
한정된 자원을 최대한 활용하여 효율적으로 하이퍼파라미터 공간을 탐색할 수 있다는 장점이 있다.

싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는데, <br>
'이를 균등 분포에서 샘플링한다'고 한다.<br>
uniform은 실숫값을, randint는 정숫값을 뽑는다.


0에서 10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링 해볼 것이다.

In [31]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([8, 3, 7, 0, 0, 3, 0, 1, 4, 6])

10개밖에 되지 않기 때문에 고르게 샘플링되는 것 같지 않지만 샘플링 숫자를 늘리면 쉽게 확인할 수 있다.<br>
1,000개를 샘플링 하여 각 숫자의 개수를 세어보자.

In [32]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102,  96, 114,  95,  97, 102, 109,  87,  95, 103]))

샘플링 수가 늘어나니 0~9 숫자가 어느 정도 고르게 추출된 것을 확인할 수 있다.<br>

uniform 클래스의 사용법도 동일하다.

In [33]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.84834399, 0.8642586 , 0.56732626, 0.66982158, 0.52838872,
       0.87910371, 0.02586952, 0.52061102, 0.02026802, 0.9025909 ])

그럼 이제 탐색할 매개변수의 딕셔너리를 만들어 볼 것이다.<br>
여기에서는 리프 노드가 되기 위한 최소 샘플의 개수인 `min_samples_leaf` 매개변수를 탐색 대상에 추가한다. <br>
어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다.

In [34]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

샘플링 횟수는 사이킷런의 랜ㄷ넘 서치 클래스인 `RandomizedSearchCV`의 `n_iter` 매개변수에 지정한다.

In [35]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f1d318e4d90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f1d31578f50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f1d318e4ad0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f1d318e4690>},
                   random_state=42)

최적의 매개변수 조합을 출력해보자.

In [36]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


최고의 교차 검증 점수도 확인해보자.

In [37]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인해보자.

In [38]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


테스트 세트 점수는 검증 세트 점수보다 조금 작은 것이 일반적이다.